## Run on Julia 1.6.0

In [1]:
using Test, PulseInputDDM, LinearAlgebra, Flatten, Parameters

In [2]:
n, cross = 53, false

(53, false)

In [3]:
## Choice model
θ = θchoice(θz=θz(σ2_i = 0.5, B = 15., λ = -0.5, σ2_a = 50., σ2_s = 1.5,
    ϕ = 0.8, τ_ϕ = 0.05),
    bias=1., lapse=0.05)

θ, data = synthetic_data(;θ=θ, ntrials=10, rng=1)
model_gen = choiceDDM(θ, data, n, cross, θprior(μ_B=40., σ_B=1e6))

choiceDDM{θchoice{θz{Float64}, Float64}, Vector{PulseInputDDM.choicedata{PulseInputDDM.choiceinputs{PulseInputDDM.clicks, PulseInputDDM.binned_clicks}}}, θprior{Float64}}
  θ: θchoice{θz{Float64}, Float64}
  data: Array{PulseInputDDM.choicedata{PulseInputDDM.choiceinputs{PulseInputDDM.clicks, PulseInputDDM.binned_clicks}}}((10,))
  n: Int64 53
  cross: Bool false
  θprior: θprior{Float64}


In [4]:
choices = getfield.(data, :choice);

In [5]:
choices

10-element BitVector:
 1
 0
 0
 0
 0
 0
 0
 0
 0
 1

In [6]:
@test all(choices .== vcat(true, falses(8), true))

Test Passed

In [7]:
@time @test round(loglikelihood(model_gen), digits=2) ≈ -3.3

  2.973292 seconds (5.27 M allocations: 315.128 MiB, 3.85% gc time, 97.06% compilation time)


Test Passed

In [8]:
@test round(norm(gradient(model_gen)), digits=2) ≈ 6.27

Test Passed

In [9]:
options = choiceoptions(lb=vcat([0., 8.,  -5., 0.,   0.,  0.01, 0.005], [-30, 0.]),
    ub = vcat([2., 30., 5., 100., 2.5, 1.2,  1.], [30, 1.]), 
    fit = trues(dimz+2));

In [10]:
model, = optimize(data, options; iterations=5, outer_iterations=1, θprior=θprior(μ_B=40., σ_B=1e6));
@test round(norm(Flatten.flatten(model.θ)), digits=2) ≈ 25.01

Fminbox
-------
Initial mu = 0.00023176

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 0.00023176

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     2.031279e+01     9.526069e+01
 * time: 0.031861066818237305

Exiting inner optimizer with x = [0.09245260117531101, 14.99534895309801, -0.07050520798299356, 19.995171687431025, 0.07683762452946646, 0.8225008941393236, 0.005001877508016104, 0.021251246288531825, 0.04094907661372967]
Current distance to box: 1.87751e-6
Decreasing barrier term μ.



Test Passed

In [11]:
H = Hessian(model)
@test round(norm(H), digits=2) ≈ 762.91

Test Passed

In [12]:
CI, HPSD = CIs(H)
@test round(norm(CI), digits=2) ≈ 587.96

┌ Warning: Hessian is not positive definite. Approximated by closest PSD matrix.
│             ||ϵ||/||H|| is 299.80355140583623
└ @ PulseInputDDM /mnt/cup/people/briandd/Projects/PulseInputDDM/src/base_model.jl:19


Test Passed

## Neural

In [13]:
## Neural model
ncells, ntrials = [1,2], [10,5]
f = [repeat(["Sigmoid"], N) for N in ncells]
                    
θ = θneural(θz = θz(σ2_i = 0.5, B = 15., λ = -0.5, σ2_a = 10., σ2_s = 1.2,
    ϕ = 0.6, τ_ϕ =  0.02),
    θy=[[Sigmoid() for n in 1:N] for N in ncells], f=f);

In [14]:
data, = synthetic_data(θ, ntrials, ncells);
model_gen = neuralDDM(θ, data, n, cross, θprior(μ_B=40., σ_B=1e6));

In [15]:
spikes = map(x-> sum.(x), getfield.(vcat(data...), :spikes))

15-element Vector{Vector{Int64}}:
 [7]
 [8]
 [5]
 [14]
 [10]
 [10]
 [9]
 [5]
 [8]
 [5]
 [4, 3]
 [9, 13]
 [11, 8]
 [1, 3]
 [11, 10]

In [16]:
@test all(spikes .== [[7], [8], [5], [14], [10], [10], [9], [5], [8], [5], [4, 3], [9, 13], [11, 8], [1, 3], [11, 10]])

Test Passed

In [17]:
@test round(loglikelihood(model_gen), digits=2) ≈ -451.56

Test Passed

In [18]:
@test round(norm(gradient(model_gen)), digits=2) ≈ 4.4

Test Passed

In [19]:
x = PulseInputDDM.flatten(θ)
@test round(loglikelihood(x, model_gen), digits=2) ≈ -451.56

Test Passed

In [20]:
θy0 = vcat(vcat(θy.(data, f)...)...)
@test round(norm(θy0), digits=2) ≈ 21.41

Test Passed

In [21]:
#deterministic model
options0 = neural_options_noiseless(f)

neural_options_noiseless
  fit: Array{Bool}((19,)) Bool[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  ub: Array{Float64}((19,)) [100.0, 100.0, 5.0, 400.0, 10.0, 1.2, 1.0, 100.0, 100.0, 10.0, 10.0, 100.0, 100.0, 10.0, 10.0, 100.0, 100.0, 10.0, 10.0]
  lb: Array{Float64}((19,)) [0.001, 8.0, -5.0, 0.001, 0.001, 0.001, 0.005, -100.0, 0.0, -10.0, -10.0, -100.0, 0.0, -10.0, -10.0, -100.0, 0.0, -10.0, -10.0]


In [22]:
x0=vcat([0., 30., 0. + eps(), 0., 0., 1. - eps(), 0.008], θy0)

19-element Vector{Float64}:
   0.0
  30.0
   2.220446049250313e-16
   0.0
   0.0
   0.9999999999999998
   0.008
  -3.27383416829478
   5.237344693493839
   0.0012036030165978307
   0.0
 -11.880907702150646
  12.523854202611258
   0.0017736028443593546
   0.0
  -7.758407900516637
   7.870264249894646
   0.0020604602058062076
   0.0

In [23]:
θ0 = θneural_noiseless(x0, f)
model0 = noiseless_neuralDDM(θ0, data)

noiseless_neuralDDM{θneural_noiseless{θz{Float64}, Vector{Vector{Sigmoid{Float64}}}}, Vector{Vector{neuraldata}}}
  θ: θneural_noiseless{θz{Float64}, Vector{Vector{Sigmoid{Float64}}}}
  data: Array{Vector{neuraldata}}((2,))


In [24]:
@test round(loglikelihood(model0), digits=2) ≈ -1127.15

Test Passed

In [25]:
x0 = PulseInputDDM.flatten(θ0)
@unpack f = θ0

θneural_noiseless{θz{Float64}, Vector{Vector{Sigmoid{Float64}}}}
  θz: θz{Float64}
  θy: Array{Vector{Sigmoid{Float64}}}((2,))
  f: Array{Vector{String}}((2,))


In [26]:
@test round(loglikelihood(x0, model0), digits=2) ≈ -1127.15

Test Passed

In [27]:
model, = optimize(model0, options0; iterations=2, outer_iterations=1)
@test round(norm(PulseInputDDM.flatten(model.θ)), digits=2) ≈ 45.21

Test Passed

In [28]:
@test round(norm(gradient(model)), digits=2) ≈ 4.64

Error During Test at In[28]:1
  Test threw exception
  Expression: round(norm(gradient(model)), digits = 2) ≈ 4.64
  DomainError with Dual{ForwardDiff.Tag{PulseInputDDM.var"#ℓℓ#224"{noiseless_neuralDDM{θneural_noiseless{θz{Float64}, Vector{Vector{Sigmoid{Float64}}}}, Vector{Vector{neuraldata}}}}, Float64}}(NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN):
  Poisson: the condition λ >= zero(λ) is not satisfied.
  Stacktrace:
    [1] (::Base.var"#837#839")(x::Task)
      @ Base ./asyncmap.jl:177
    [2] foreach(f::Base.var"#837#839", itr::Vector{Any})
      @ Base ./abstractarray.jl:2141
    [3] maptwice(wrapped_f::Function, chnl::Channel{Any}, worker_tasks::Vector{Any}, c::Vector{neuraldata})
      @ Base ./asyncmap.jl:177
    [4] wrap_n_exec_twice
      @ ./asyncmap.jl:153 [inlined]
    [5] #async_usemap#822
      @ ./asyncmap.jl:103 [inlined]
    [6] #asyncmap#821
      @ ./asyncmap.jl:81 [inlined]
    [7] pmap(f::Function, p::Distributed.WorkerPool, c::Vector{neuraldata}; distributed::Bo

LoadError: [91mThere was an error during testing[39m

In [29]:
x0 = vcat([0.1, 15., -0.1, 20., 0.5, 0.8, 0.008], PulseInputDDM.flatten(model.θ)[dimz+1:end])

19-element Vector{Float64}:
  0.1
 15.0
 -0.1
 20.0
  0.5
  0.8
  0.008
 17.071225279527855
 12.640373768299604
 -4.011715642283551
  6.102566985248116
 -2.804143285567207
 19.085462303603528
  2.673281795601461
  0.9550686995832421
  2.994916427273023
 15.645197993480087
  1.9258252163343177
  0.8149348291364622

In [30]:
options = neural_options(f)  

model = neuralDDM(θneural(x0, f), data, n, cross, θprior(μ_B=40., σ_B=1e6))
model, = optimize(model, options; iterations=2, outer_iterations=1)
@test round(norm(PulseInputDDM.flatten(model.θ)), digits=2) ≈ 41.17

Fminbox
-------
Initial mu = 0.000549494

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 0.000549494

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     4.797230e+02     1.703780e+02
 * time: 2.002716064453125e-5

Exiting inner optimizer with x = [1.0530076742670165, 15.077956768838995, 1.5128449716893115, 20.092381014683916, 0.9428569132520415, 0.7524259282726876, 0.6817275699393194, 14.659833117070193, 11.661694475460566, -5.601499616526606, 5.075616661274729, -2.368940954266793, 19.41962071534353, 2.5078212655054872, 1.661312401384596, 3.068854594327254, 15.680560718218166, 1.7794878641685616, 0.6534128629270439]
Current distance to box: 0.318272
Decreasing barrier term μ.



Test Passed

In [31]:
H = Hessian(model; chunk_size=4)
@test round(norm(H), digits=2) ≈ 9.17

Test Passed

In [32]:
CI, HPSD = CIs(H)
@test round(norm(CI), digits=2) ≈ 917.8

┌ Warning: Hessian is not positive definite. Approximated by closest PSD matrix.
│             ||ϵ||/||H|| is 0.5547076975499954
└ @ PulseInputDDM /mnt/cup/people/briandd/Projects/PulseInputDDM/src/base_model.jl:19


Test Passed

In [33]:
options = neural_choice_options(f)

neural_choice_options
  fit: Array{Bool}((21,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  ub: Array{Float64}((21,)) [100.0, 40.0, 5.0, 400.0, 10.0, 1.2, 1.0, 10.0, 1.0, 100.0  …  10.0, 10.0, 100.0, 100.0, 10.0, 10.0, 100.0, 100.0, 10.0, 10.0]
  lb: Array{Float64}((21,)) [0.001, 8.0, -5.0, 0.001, 0.001, 0.001, 0.005, -10.0, 0.0, -100.0  …  -10.0, -10.0, -100.0, 0.0, -10.0, -10.0, -100.0, 0.0, -10.0, -10.0]


In [34]:
choice_neural_model = neural_choiceDDM(θneural_choice(vcat(x0[1:dimz], 0., 0., x0[dimz+1:end]), f), data, n, cross)

neural_choiceDDM{θneural_choice{θz{Float64}, Float64, Vector{Vector{Sigmoid{Float64}}}}, Vector{Vector{neuraldata}}}
  θ: θneural_choice{θz{Float64}, Float64, Vector{Vector{Sigmoid{Float64}}}}
  data: Array{Vector{neuraldata}}((2,))
  n: Int64 53
  cross: Bool false


In [35]:
@test round(choice_loglikelihood(choice_neural_model), digits=2) ≈ -6.45

Test Passed

In [36]:
@test round(joint_loglikelihood(choice_neural_model), digits=2) ≈ -486.23

Test Passed

In [37]:
import PulseInputDDM: nθparams
nparams, = nθparams(f)

([4, 4, 4], [1, 2])

In [38]:
fit = vcat(falses(dimz), trues(2), falses.(nparams)...);
options = neural_choice_options(fit=fit, lb=options.lb, ub=options.ub)

neural_choice_options
  fit: Array{Bool}((21,)) Bool[0, 0, 0, 0, 0, 0, 0, 1, 1, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  ub: Array{Float64}((21,)) [100.0, 40.0, 5.0, 400.0, 10.0, 1.2, 1.0, 10.0, 1.0, 100.0  …  10.0, 10.0, 100.0, 100.0, 10.0, 10.0, 100.0, 100.0, 10.0, 10.0]
  lb: Array{Float64}((21,)) [0.001, 8.0, -5.0, 0.001, 0.001, 0.001, 0.005, -10.0, 0.0, -100.0  …  -10.0, -10.0, -100.0, 0.0, -10.0, -10.0, -100.0, 0.0, -10.0, -10.0]


In [39]:
choice_neural_model, = choice_optimize(choice_neural_model, options; iterations=2, outer_iterations=1)

┌ Warning: Initial position cannot be on the boundary of the box. Moving elements to the interior.
│ Element indices affected: [2]
└ @ Optim /usr/people/briandd/.julia/packages/Optim/tP8PJ/src/multivariate/solvers/constrained/fminbox.jl:314


Fminbox
-------
Initial mu = 0.000560534

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 0.000560534

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     5.530525e+00     5.617773e+01
 * time: 1.7881393432617188e-5

Exiting inner optimizer with x = [0.23159355604200516, 0.3782997206127649]
Current distance to box: 0.3783
Decreasing barrier term μ.



(neural_choiceDDM{θneural_choice{θz{Float64}, Float64, Vector{Vector{Sigmoid{Float64}}}}, Vector{Vector{neuraldata}}}
  θ: θneural_choice{θz{Float64}, Float64, Vector{Vector{Sigmoid{Float64}}}}
  data: Array{Vector{neuraldata}}((2,))
  n: Int64 53
  cross: Bool false
,  * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     5.010782e+00

 * Found with
    Algorithm:     Fminbox with BFGS

 * Convergence measures
    |x - x'|               = 4.35e-01 ≰ 1.0e-10
    |x - x'|/|x'|          = 9.81e-01 ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 1.0e-09
    |g(x)|                 = 3.78e-01 ≰ 1.0e-03

 * Work counters
    Seconds run:   7  (vs limit 170000)
    Iterations:    1
    f(x) calls:    4
    ∇f(x) calls:   3
)

In [40]:
@test round(norm(PulseInputDDM.flatten(choice_neural_model.θ)), digits=2) ≈ 42.06

Test Passed

In [41]:
choice_neural_model = neural_choiceDDM(θneural_choice(vcat(x0[1:dimz], 0., 0., x0[dimz+1:end]), f), data, n, cross)

neural_choiceDDM{θneural_choice{θz{Float64}, Float64, Vector{Vector{Sigmoid{Float64}}}}, Vector{Vector{neuraldata}}}
  θ: θneural_choice{θz{Float64}, Float64, Vector{Vector{Sigmoid{Float64}}}}
  data: Array{Vector{neuraldata}}((2,))
  n: Int64 53
  cross: Bool false


In [42]:
fit = vcat(trues(dimz), trues(2), trues.(nparams)...);
options = neural_choice_options(fit=fit, lb=vcat(options.lb[1:7], -10., options.lb[9:end]), 
    ub=vcat(options.ub[1:7], 10., options.ub[9:end]))

neural_choice_options
  fit: Array{Bool}((21,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  ub: Array{Float64}((21,)) [100.0, 40.0, 5.0, 400.0, 10.0, 1.2, 1.0, 10.0, 1.0, 100.0  …  10.0, 10.0, 100.0, 100.0, 10.0, 10.0, 100.0, 100.0, 10.0, 10.0]
  lb: Array{Float64}((21,)) [0.001, 8.0, -5.0, 0.001, 0.001, 0.001, 0.005, -10.0, 0.0, -100.0  …  -10.0, -10.0, -100.0, 0.0, -10.0, -10.0, -100.0, 0.0, -10.0, -10.0]


In [43]:
choice_neural_model, = choice_optimize(choice_neural_model, options; iterations=2, outer_iterations=1)

┌ Warning: Initial position cannot be on the boundary of the box. Moving elements to the interior.
│ Element indices affected: [9]
└ @ Optim /usr/people/briandd/.julia/packages/Optim/tP8PJ/src/multivariate/solvers/constrained/fminbox.jl:314


Fminbox
-------
Initial mu = 0.000180107

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 0.000180107

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     5.514275e+00     5.619398e+01
 * time: 1.7881393432617188e-5

Exiting inner optimizer with x = [0.09999268868157225, 15.000000413966145, -0.1000409940028152, 19.999999687486646, 0.4998675886289066, 0.8001149204296798, 0.00500780905147053, 7.092469212054133e-5, 0.0169013086762203, 17.07122187605948, 12.640354134521216, -4.011712506688259, 6.102569142823409, -2.8041338797149864, 19.085466449710562, 2.673279937226764, 0.9550796532172342, 2.994927363012368, 15.64520015227081, 1.9258198981150547, 0.8149403419493705]
Current distance to box: 7.80905e-6
Decreasing barrier term μ.



(neural_choiceDDM{θneural_choice{θz{Float64}, Float64, Vector{Vector{Sigmoid{Float64}}}}, Vector{Vector{neuraldata}}}
  θ: θneural_choice{θz{Float64}, Float64, Vector{Vector{Sigmoid{Float64}}}}
  data: Array{Vector{neuraldata}}((2,))
  n: Int64 53
  cross: Bool false
,  * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     5.139852e+00

 * Found with
    Algorithm:     Fminbox with BFGS

 * Convergence measures
    |x - x'|               = 7.52e-03 ≰ 1.0e-10
    |x - x'|/|x'|          = 1.79e-04 ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 1.0e-09
    |g(x)|                 = 9.83e-01 ≰ 1.0e-03

 * Work counters
    Seconds run:   8  (vs limit 170000)
    Iterations:    1
    f(x) calls:    3
    ∇f(x) calls:   3
)

In [44]:
@test round(norm(PulseInputDDM.flatten(choice_neural_model.θ)), digits=2) ≈ 42.06

Test Passed